In [1]:
import os
import openai
import langchain
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader

In [ ]:
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-05-15"
os.environ["AZURE_OPENAI_ENDPOINT"] = "Your Endpoint"
os.environ["OPENAI_API_KEY"] = "Your API Key"

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA

In [4]:
class AzureOpenAIEmbeddings:
    def __init__(self, deployment_name, api_key, api_base, api_version):
        self.deployment_name = deployment_name
        self.api_key = api_key
        self.api_base = api_base
        self.api_version = api_version
        openai.api_type = "azure"
        openai.api_key = self.api_key
        openai.api_base = self.api_base
        openai.api_version = self.api_version

    def embed_documents(self, texts):
        # Use batch processing
        response = openai.Embedding.create(
            input=texts,
            deployment_id=self.deployment_name  # Use deployment_id for Azure OpenAI
        )
        embeddings = [item['embedding'] for item in response['data']]
        return embeddings


In [ ]:
azure_embeddings = AzureOpenAIEmbeddings(
    deployment_name="Your deployment name",  # Replace with your deployment name
    api_key=os.getenv("your api key"),
    api_base=os.getenv("your api base url"),
    api_version="2023-05-15"
)

In [7]:
# Load and split the PDF document
loader = PyPDFLoader("Customer Information.pdf")
pages = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [8]:
print(texts)

[Document(metadata={'source': 'Customer Information.pdf', 'page': 0}, page_content="Customer Information  \n \nName: Rohan Sharma \nAge: 32 years \nGender: Male \nLocation: Bengaluru, Karnataka, India \nOccupation: Software Engineer \nIncome Level: ₹1,200,000 per year \nEducation: Bachelor's degree in Computer Science \nLanguage Preferences: English, Hindi \nPAN: YINO12E4")]


In [ ]:
from chromadb import Chroma
from azure_embeddings import AzureEmbeddings  # Ensure you have the correct implementation for Azure embeddings

# Initialize Azure embeddings instance
azure_embeddings = AzureEmbeddings(
    deployment_name="your-deployment-name",  # Replace with your deployment name
    api_key="your-api-key",  # Replace with your API key
    api_base="your-api-base-url",  # Replace with your Azure endpoint
    api_version="2023-05-15"  # Replace with the correct API version
)

# Assuming you've already generated embeddings for your document chunks
document_embeddings = azure_embeddings.embed_documents([text.page_content for text in texts])

# Initialize Chroma vector store
vector_store = Chroma.from_embeddings(
    embeddings=document_embeddings,  # List of embeddings
    texts=[text.page_content for text in texts],  # Corresponding list of text content
    embedding_function=azure_embeddings.embed_query  # Function to use for querying
)


ImportError: cannot import name 'Chroma' from 'chromadb' (c:\Users\Sivap\Documents\SRM notes\NLP\Gen-AI Project\.venv\Lib\site-packages\chromadb\__init__.py)

In [ ]:
# Generate embeddings for the document chunks
document_embeddings = azure_embeddings.embed_documents([text.page_content for text in texts])

In [ ]:
# Initialize Chroma vector store with the generated embeddings
vector_store = Chroma.from_embeddings(
    embeddings=document_embeddings,
    texts=[text['text'] for text in texts],
    embedding_function=azure_embeddings.embed_query  # Use for query embedding

In [ ]:
# Create a QA chain with VectorDBQA
qa_chain = VectorDBQA.from_chain_type(
    vector_store=vector_store,
    embedding_function=azure_embeddings.embed_query

In [ ]:
query = "Enter your question here"
response = qa_chain.run(query)
print("Response:", response)